# [Finding a Learning Rate with Tensorflow 2](https://www.avanwyk.com/finding-a-learning-rate-in-tensorflow-2/)

In [ ]:
import tensorflow as tf

In [ ]:
# 當 library or module 改變，會自動重新載入
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_valid, y_valid) = fashion_mnist.load_data()
x_train, x_valid = x_train / 255.0, x_valid / 255.0

x_train = x_train[..., tf.newaxis]
x_valid = x_valid[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(32)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

def build_model():
    return tf.keras.models.Sequential([
        Conv2D(32, 3, activation='relu'),
        MaxPool2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.1),
        Dense(10, activation='softmax')
    ])

In [ ]:
#載入 lr_finder.py 檔
from lr_finder import LRFinder

In [ ]:
lr_finder = LRFinder()
model = build_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
_ = model.fit(train_ds, epochs=5, callbacks=[lr_finder], verbose=1)
lr_finder.plot()

import matplotlib.pyplot as plt
plt.axvline(1e-3, c='r')

In [ ]:
model.evaluate(valid_ds, verbose=False)

## 採用使最佳學習率

In [ ]:
# lr should be chosen in a region where the loss is rapidly, but strictly decreasing.
model = build_model() # reinitialize model
adam = tf.optimizers.Adam(1e-3)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
_ = model.fit(train_ds, validation_data=valid_ds, epochs=5, verbose=True)

In [ ]:
model.evaluate(valid_ds, verbose=False)